In [ ]:
!pip install pandas
!pip install tqdm
!pip install tenacity
!pip install beautifulsoup4
!pip install nltk

## Gathering the Data
We'll be using the [congress.gov API](https://github.com/LibraryOfCongress/api.congress.gov) to request all of the data on bills introduced in the House of Representatives during the 117th Congress. 

In [41]:
from cdg_client import CDGClient

with open("congress_api.env", "r") as f:
    api_key = f.read()

client = CDGClient(api_key)

In [42]:
# Get information on HR 3684
example_bill = client.get("bill/117/hr/3684")
print(example_bill[0].get("bill").get("title"))

Infrastructure Investment and Jobs Act


In [1]:
import json
from tqdm import tqdm
from tqdm.contrib.logging import logging_redirect_tqdm
from requests.exceptions import HTTPError
from tenacity import retry, wait_fixed, retry_if_exception_type, before_sleep_log
import logging
import sys
import requests
from time import sleep

C:\Users\pramo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [39]:
logging.basicConfig(stream=sys.stderr, level=logging.DEBUG)
logger = logging.getLogger(__name__)

# suppress debug logs from urllib3
logging.getLogger("urllib3").setLevel(logging.WARNING)

# custom logging function that logs retries
def retry_log(logger, level):
    def wrapper(retry_state):
        if retry_state.attempt_number > 1:
            logger.log(level, f"Retrying {retry_state.fn.__name__} (attempt {retry_state.attempt_number})")
    return wrapper

# wait 5 minutes and try again if an exception is raised (rate limited to 5000 API calls per hour)
@retry(wait=wait_fixed(300), retry=retry_if_exception_type(HTTPError), before_sleep=retry_log(logger, logging.DEBUG))
def getBill(congress, chamber, number):
    return client.get(f"bill/{congress}/{chamber}/{number}")

@retry(wait=wait_fixed(300), retry=retry_if_exception_type(HTTPError), before_sleep=retry_log(logger, logging.DEBUG))
def getBillText(congress, chamber, number):
    return client.get(f"bill/{congress}/{chamber}/{number}/text")

@retry(wait=wait_fixed(300), retry=retry_if_exception_type(HTTPError), before_sleep=retry_log(logger, logging.DEBUG))
def getBillSummary(congress, chamber, number):
    return client.get(f"bill/{congress}/{chamber}/{number}/summaries")

@retry(wait=wait_fixed(300), retry=retry_if_exception_type(HTTPError), before_sleep=retry_log(logger, logging.DEBUG))
def getBillActions(congress, chamber, number):
    return client.get(f"bill/{congress}/{chamber}/{number}/actions")

In [5]:
NUM_BILLS = 9709# 9709 bills were introduced by the House of the 117th congress

start = 1

with logging_redirect_tqdm():
    for i in tqdm(range(start, NUM_BILLS + 1)):
        bill = getBill(117, "hr", i)    

        with open(f"BillData/JSONFiles/{str(i).zfill(5)}.json", "w") as outfile: 
            json.dump(bill[0].get("bill"), outfile)

 30%|██▉       | 2872/9709 [15:00<35:30,  3.21it/s]  

In [39]:
NUM_BILLS = 9709
start = 1

with logging_redirect_tqdm():
    for i in tqdm(range(start, NUM_BILLS + 1)):
        sources = getBillText(117, "hr", i)

        try:
            formats = sources[0].get("textVersions")[0].get("formats")
        except IndexError:
            print(f"Failed to retrieve the content. No text is available for H.R. {i}.")
            continue

        url = next((item["url"] for item in formats if item["type"] == "Formatted Text"), None)

        response = requests.get(url)
        if response.status_code == 200:
            with open(f"BillData/BillText/{str(i).zfill(5)}.htm", "w", encoding="utf-8") as file:
                file.write(response.text)
        else:
            print(f"Failed to retrieve the content. Status code: {response.status_code}")
        
        # wait before sending more https requests
        sleep(2)

  0%|          | 2/9709 [00:02<3:03:45,  1.14s/it]

Failed to retrieve the content. No text is available for H.R. 2.


  0%|          | 9/9709 [00:16<4:18:02,  1.60s/it]

Failed to retrieve the content. No text is available for H.R. 9.


  0%|          | 10/9709 [00:16<3:09:02,  1.17s/it]

...

  0%|          | 17/9709 [00:18<46:26,  3.48it/s]

Failed to retrieve the content. No text is available for H.R. 17.


  0%|          | 20/9709 [00:23<2:38:19,  1.02it/s]

Failed to retrieve the content. No text is available for H.R. 20.


100%|██████████| 9709/9709 [6:39:28<00:00,  2.47s/it]  


In [6]:
NUM_BILLS = 9709
start = 4944

with logging_redirect_tqdm():
    for i in tqdm(range(start, NUM_BILLS + 1)):
        response = getBillSummary(117, "hr", i)[0].get("summaries")

        if len(response) == 0:
            logging.debug(f"Failed to retrieve the content. No summary is available for H.R. {i}")
            continue

        summary = response[0].get("text")
        
        with open(f"BillData/BillSummaries/{str(i).zfill(5)}.htm", "w", encoding="utf-8") as file:
            file.write(summary)
        
        sleep(1.4)

100%|██████████| 4766/4766 [2:10:04<00:00,  1.64s/it]  


In [4]:
getBillSummary(117, "hr", 4944)[0].get("summaries")[0]

{'actionDate': '2021-08-06',
 'actionDesc': 'Introduced in House',
 'text': " <p><strong>Helping Kids Cope Act of 2021</strong></p> <p>This bill provides funding through FY2026 to the Health Resources and Services Administration (HRSA) for grants to enhance access to and provider training in pediatric behavioral health care.</p> <p>Specifically, HRSA must award grants to pediatricians, children's hospitals, and other providers to support pediatric behavioral health integration and coordination within communities. Grant-funded activities may include hiring community navigators to assist families in accessing appropriate services, incorporating behavioral health services in pediatric practices, and delivering services via telehealth.</p> <p>In addition, HRSA must award grants to children's hospitals to expand training for providers in the pediatric behavioral health workforce.</p>",
 'updateDate': '2022-02-03T22:40:03Z',
 'versionCode': '00'}

In [45]:
getBillActions(117, "hr", 1)  

({'actions': [{'actionDate': '2021-03-11',
    'sourceSystem': {'name': 'Senate'},
    'text': 'Received in the Senate.',
    'type': 'IntroReferral'},
   {'actionCode': 'H38900',
    'actionDate': '2021-03-03',
    'actionTime': '23:05:48',
    'sourceSystem': {'code': 2, 'name': 'House floor actions'},
    'text': 'The Clerk was authorized to correct section numbers, punctuation, and cross references, and to make other necessary technical and conforming corrections in the engrossment of H.R. 1.',
    'type': 'Floor'},
   {'actionCode': 'H38310',
    'actionDate': '2021-03-03',
    'actionTime': '23:05:39',
    'sourceSystem': {'code': 2, 'name': 'House floor actions'},
    'text': 'Motion to reconsider laid on the table Agreed to without objection.',
    'type': 'Floor'},
   {'actionCode': 'H37100',
    'actionDate': '2021-03-03',
    'actionTime': '23:05:36',
    'recordedVotes': [{'chamber': 'House',
      'congress': 117,
      'date': '2021-03-04T04:05:36Z',
      'rollNumber': 6

In [ ]:
NUM_BILLS = 9709
start = 246

with logging_redirect_tqdm():
    for i in tqdm(range(start, NUM_BILLS + 1)):
        actions = getBillActions(117, "hr", i)    

        with open(f"BillData/BillActions/{str(i).zfill(5)}.json", "w") as outfile: 
            json.dump(actions, outfile)
        
        sleep(0.5)

## Assembling the dataset

In [2]:
import os
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
import re
from pathlib import Path

In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pramo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pramo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [34]:
# get pretrained GloVe model - Common Crawl (42B tokens, 1.9M vocab, uncased, 300d vectors, 1.75 GB download)
!wget https://huggingface.co/stanfordnlp/glove/resolve/main/glove.42B.300d.zip
!tar -xf glove.42B.300d.zip

--2024-07-21 11:48:26--  https://huggingface.co/stanfordnlp/glove/resolve/main/glove.42B.300d.zip
Resolving huggingface.co... 18.160.102.86, 18.160.102.96, 18.160.102.4, ...
Connecting to huggingface.co|18.160.102.86|:443... connected.
OpenSSL: error:140773E8:SSL routines:SSL23_GET_SERVER_HELLO:reason(1000)
Unable to establish SSL connection.
tar: Error opening archive: Failed to open 'glove.42B.300d.zip'


In [4]:
def get_clean_bill_summary(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')

        # Remove <b> and <strong> tags (these tags contain the bill title)
        for tag in ['b', 'strong']:
            for match in soup.find_all(tag):
                match.decompose()

        # Convert to string and lowercase
        summary = " ".join(soup.stripped_strings)
        summary = summary.lower()

        # Drop all punctuation
        summary = re.sub(r"[^\w\s]", "", summary)
        
    return summary

def get_clean_bill_title(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        data = json.load(file)

        title = data["title"].lower()
        title = re.sub(r"[^\w\s]", "", title)

    return title

def tokenize_text(text):
    # tokenize text
    tokens = nltk.word_tokenize(text.lower())

    # get rid of stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    return tokens

# example usage
get_clean_bill_summary("BillData/BillSummaries/00033.htm")


'this bill abolishes the agency for toxic substances and disease registry this agency which is part of the department of health and human services hhs investigates and responds to environmental exposures to hazardous substances in communities hhs must transfer authority regarding certain national disease and toxic exposure registries to another appropriate hhs entity'

In [5]:
# function to test if a bill passed house
def did_bill_pass(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        if "Senate" in file.read():
            return 1
        else:
            return 0

# example usage
did_bill_pass("BillData/BillActions/07776.json")

1

In [23]:
# testing to see how many bills are missing the "policyArea" attribute

total = 0
count = 0

for file in tqdm(Path("BillData/BillSummaries").glob('*.htm')):
    bill_number = str(file).split("\\")[-1][:-4]

    with open(f"BillData/JSONFiles/{bill_number}.json", "r", encoding="utf-8") as file:
        data = json.load(file)

        try:
            policy_area = data["policyArea"]["name"]
        except KeyError:
            count += 1
        
        total +=1

print(f"{count}/{total}={count/total}")
            

9698it [00:50, 192.01it/s]

29/9698=0.002990307279851516


In [24]:
features = ["bill_id", 
            "title", 
            "summary", 
            "sponsor_party", 
            "sponsor_state",
            "sponsor_district",
            "policy_area", 
            "subjects_count", 
            "introduced_date",
            "passed_house"]

dataset = pd.DataFrame(columns = features)

# iterate through htm files in the BillSummaries directory
for file in tqdm(Path("BillData/BillSummaries").glob('*.htm')):
    bill_number = str(file).split("\\")[-1][:-4]
    
    summary = get_clean_bill_summary(file)
    summary = tokenize_text(summary)

    # remove word "bill" if it is first word
    if summary[0] == "bill":
        summary.pop(0)

    with open(f"BillData/JSONFiles/{bill_number}.json", "r", encoding="utf-8") as file:
        data = json.load(file)
        
        title = data["title"].lower()
        title = re.sub(r"[^\w\s]", "", title)
        title = tokenize_text(title)

        sponsor_party = data["sponsors"][0]["party"]
        sponsor_state = data["sponsors"][0]["state"]
        
        introduced_date = data["introducedDate"]


        passed_house = did_bill_pass(f"BillData/BillActions/{bill_number}.json")

        # 5 bills are missing subject count
        try:
            subjects_count = data["subjects"]["count"]
        except Exception:
            subjects_count = 1

        # 29 bills are missing the policyArea attribute
        try:
            policy_area = data["policyArea"]["name"]
        except KeyError:
            policy_area = "unknown"

        # some regions only have one district
        # if the bill is missing a sponsor district, it's likely the bill is from such a region
        try:
            sponsor_district = data["sponsors"][0]["district"]
        except KeyError:
            sponsor_district = 1
        
    
    # add the row to the dataset
    new_row = pd.DataFrame([[int(bill_number),
                                 title,
                                 summary,
                                 sponsor_party,
                                 sponsor_state,
                                 sponsor_district,
                                 policy_area,
                                 subjects_count,
                                 introduced_date,
                                 passed_house]], columns = features)
    
    dataset = pd.concat([dataset, new_row])

dataset

9698it [00:24, 403.67it/s]


,bill_id,title,summary,sponsor_party,sponsor_state,sponsor_district,policy_area,subjects_count,introduced_date,passed_house
0,1,"[people, act, 2021]","[addresses, voter, access, election, integrity...",D,MD,3,Government Operations and Politics,113,2021-01-04,1
0,3,"[elijah, e, cummings, lower, drug, costs, act]","[establishes, several, programs, requirements,...",D,NJ,6,Health,47,2021-04-22,0
0,4,"[john, r, lewis, voting, rights, advancement, ...","[establishes, new, criteria, determining, stat...",D,AL,7,Government Operations and Politics,10,2021-08-17,1
0,5,"[equality, act]","[prohibits, discrimination, based, sex, sexual...",D,RI,1,"Civil Rights and Liberties, Minority Issues",15,2021-02-18,1
0,6,"[american, dream, promise, act, 2021]","[provides, certain, aliens, path, receive, per...",D,CA,40,Immigration,39,2021-03-03,1
...,...,...,...,...,...,...,...,...,...,...
0,9705,"[promoting, diverse, investment, advisers, act]","[requires, federal, reserve, board, set, goals...",D,OH,3,Finance and Financial Sector,1,2023-01-03,0
0,9706,"[ensuring, diverse, leadership, act, 2022]","[requires, appointing, president, federal, res...",D,OH,3,Finance and Financial Sector,1,2023-01-03,0
0,9707,"[mom, act]","[provides, block, grants, states, provide, chi...",R,TX,1,Families,1,2023-01-03,0
0,9708,"[fixing, administrations, unethical, corrupt, ...","[prohibits, specified, federal, officials, emp...",R,TX,1,Crime and Law Enforcement,1,2023-01-03,0


In [27]:
# save the dataframe
dataset.to_pickle("117hrbills.pkl")
dataset.to_csv("117hrbills.csv", index=False)

In [28]:
dataset = pd.read_pickle("117hrbills.pkl")

In [36]:
# one hot encode these columns
dataset_encoded = pd.get_dummies(dataset, columns=["sponsor_party", "sponsor_state", "policy_area"])

# convert dates to datetime format
dataset_encoded["introduced_date"] = pd.to_datetime(dataset_encoded["introduced_date"], format="%Y-%m-%d")

dataset_encoded

,bill_id,title,summary,sponsor_district,subjects_count,introduced_date,passed_house,sponsor_party_D,sponsor_party_R,sponsor_state_AK,...,policy_area_Private Legislation,policy_area_Public Lands and Natural Resources,"policy_area_Science, Technology, Communications",policy_area_Social Sciences and History,policy_area_Social Welfare,policy_area_Sports and Recreation,policy_area_Taxation,policy_area_Transportation and Public Works,policy_area_Water Resources Development,policy_area_unknown
0,1,"[people, act, 2021]","[addresses, voter, access, election, integrity...",3,113,2021-01-04,1,True,False,False,...,False,False,False,False,False,False,False,False,False,False
0,3,"[elijah, e, cummings, lower, drug, costs, act]","[establishes, several, programs, requirements,...",6,47,2021-04-22,0,True,False,False,...,False,False,False,False,False,False,False,False,False,False
0,4,"[john, r, lewis, voting, rights, advancement, ...","[establishes, new, criteria, determining, stat...",7,10,2021-08-17,1,True,False,False,...,False,False,False,False,False,False,False,False,False,False
0,5,"[equality, act]","[prohibits, discrimination, based, sex, sexual...",1,15,2021-02-18,1,True,False,False,...,False,False,False,False,False,False,False,False,False,False
0,6,"[american, dream, promise, act, 2021]","[provides, certain, aliens, path, receive, per...",40,39,2021-03-03,1,True,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,9705,"[promoting, diverse, investment, advisers, act]","[requires, federal, reserve, board, set, goals...",3,1,2023-01-03,0,True,False,False,...,False,False,False,False,False,False,False,False,False,False
0,9706,"[ensuring, diverse, leadership, act, 2022]","[requires, appointing, president, federal, res...",3,1,2023-01-03,0,True,False,False,...,False,False,False,False,False,False,False,False,False,False
0,9707,"[mom, act]","[provides, block, grants, states, provide, chi...",1,1,2023-01-03,0,False,True,False,...,False,False,False,False,False,False,False,False,False,False
0,9708,"[fixing, administrations, unethical, corrupt, ...","[prohibits, specified, federal, officials, emp...",1,1,2023-01-03,0,False,True,False,...,False,False,False,False,False,False,False,False,False,False


In [40]:
# start and end dates of 117th congress
congress_start_date = dataset_encoded["introduced_date"].min()
congress_end_date = dataset_encoded["introduced_date"].max()

print(f"Congress Start Date: {congress_start_date}")
print(f"Congress Start Date: {congress_end_date}")

Congress Start Date: 2021-01-04 00:00:00
Congress Start Date: 2023-01-03 00:00:00


In [41]:

# Calculate the total number of days in the congressional session
total_days = (congress_end_date - congress_start_date).days

# function to normalize the dates between 0 and 1 where 0 is the start of congress and 1 is the end date
def normalize_date(date, start_date, total_days):
    return (date - start_date).days / total_days

# Apply the normalization function
dataset_encoded['introduced_date'] = dataset_encoded['introduced_date'].apply(normalize_date, args=(congress_start_date, total_days))

dataset_encoded

,bill_id,title,summary,sponsor_district,subjects_count,introduced_date,passed_house,sponsor_party_D,sponsor_party_R,sponsor_state_AK,...,policy_area_Private Legislation,policy_area_Public Lands and Natural Resources,"policy_area_Science, Technology, Communications",policy_area_Social Sciences and History,policy_area_Social Welfare,policy_area_Sports and Recreation,policy_area_Taxation,policy_area_Transportation and Public Works,policy_area_Water Resources Development,policy_area_unknown
0,1,"[people, act, 2021]","[addresses, voter, access, election, integrity...",3,113,0.000000,1,True,False,False,...,False,False,False,False,False,False,False,False,False,False
0,3,"[elijah, e, cummings, lower, drug, costs, act]","[establishes, several, programs, requirements,...",6,47,0.148148,0,True,False,False,...,False,False,False,False,False,False,False,False,False,False
0,4,"[john, r, lewis, voting, rights, advancement, ...","[establishes, new, criteria, determining, stat...",7,10,0.308642,1,True,False,False,...,False,False,False,False,False,False,False,False,False,False
0,5,"[equality, act]","[prohibits, discrimination, based, sex, sexual...",1,15,0.061728,1,True,False,False,...,False,False,False,False,False,False,False,False,False,False
0,6,"[american, dream, promise, act, 2021]","[provides, certain, aliens, path, receive, per...",40,39,0.079561,1,True,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,9705,"[promoting, diverse, investment, advisers, act]","[requires, federal, reserve, board, set, goals...",3,1,1.000000,0,True,False,False,...,False,False,False,False,False,False,False,False,False,False
0,9706,"[ensuring, diverse, leadership, act, 2022]","[requires, appointing, president, federal, res...",3,1,1.000000,0,True,False,False,...,False,False,False,False,False,False,False,False,False,False
0,9707,"[mom, act]","[provides, block, grants, states, provide, chi...",1,1,1.000000,0,False,True,False,...,False,False,False,False,False,False,False,False,False,False
0,9708,"[fixing, administrations, unethical, corrupt, ...","[prohibits, specified, federal, officials, emp...",1,1,1.000000,0,False,True,False,...,False,False,False,False,False,False,False,False,False,False


In [42]:
dataset_encoded.columns

Index(['bill_id', 'title', 'summary', 'sponsor_district', 'subjects_count',
       'introduced_date', 'passed_house', 'sponsor_party_D', 'sponsor_party_R',
       'sponsor_state_AK', 'sponsor_state_AL', 'sponsor_state_AR',
       'sponsor_state_AS', 'sponsor_state_AZ', 'sponsor_state_CA',
       'sponsor_state_CO', 'sponsor_state_CT', 'sponsor_state_DC',
       'sponsor_state_DE', 'sponsor_state_FL', 'sponsor_state_GA',
       'sponsor_state_GU', 'sponsor_state_HI', 'sponsor_state_IA',
       'sponsor_state_ID', 'sponsor_state_IL', 'sponsor_state_IN',
       'sponsor_state_KS', 'sponsor_state_KY', 'sponsor_state_LA',
       'sponsor_state_MA', 'sponsor_state_MD', 'sponsor_state_ME',
       'sponsor_state_MI', 'sponsor_state_MN', 'sponsor_state_MO',
       'sponsor_state_MP', 'sponsor_state_MS', 'sponsor_state_MT',
       'sponsor_state_NC', 'sponsor_state_ND', 'sponsor_state_NE',
       'sponsor_state_NH', 'sponsor_state_NJ', 'sponsor_state_NM',
       'sponsor_state_NV', 'sponsor_stat

In [48]:
# save the encoded dataset
dataset_encoded.to_pickle("117hrbills_encoded.pkl")
dataset_encoded.to_csv("117hrbills_encoded.csv", index=False)